# Automation for Peer Review Process

#### Necessary Changes To Be Made

In the 2nd cell start_date, end_date, month, year, week and percent values must be changed.
Also remove member name from the "reviewers" list if member data is not present.

In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import random
import glob

In [2]:
#As peer review is performed weekly, individual member files are kept in respective month and week folder
#dates in mmddyyyy format
start_date="01272020"
end_date="01312020"
month="January"
year="2020"
week="week5"
percent=0.2 #percent of data to be peer reviewed
file_name=start_date+'_'+end_date
path="../"+month+"_"+year+"/"+week
reviewers=['SP','GP','PK','MA']
print(path)

../January_2020/week5


In [5]:
glob.glob(path+'/'+week+'*.xlsx')

['../January_2020/week5\\week5_GP.xlsx',
 '../January_2020/week5\\week5_MA.xlsx',
 '../January_2020/week5\\week5_PK.xlsx',
 '../January_2020/week5\\week5_SP.xlsx']

In [ ]:
#Appending individual member data and creating a compiled csv file
MTM_data = pd.DataFrame()
for f in glob.glob(path+'/'+week+'*.xlsx'):
    member_data = pd.read_excel(f,index_col = None)
    MTM_data = MTM_data.append(member_data,  ignore_index=True, sort=False)
writer = pd.ExcelWriter(path+'/'+file_name+'.xlsx', engine='xlsxwriter')
MTM_data.to_csv(path+'/'+file_name+'.csv', encoding='utf-8')
print('Dimensions of MTM_data:',MTM_data.shape)

In [ ]:
#get the column names of the data
MTM_data

In [ ]:
#Splitting the Matched column on space to get the exact status of the title
MTM_data['Status'] = MTM_data['Matches'].str.split(' ').str[0]
MTM_data.head()

In [ ]:
#checking the unique values in the status column
MTM_data['Status'].unique()

In [ ]:
#Changing the names of status as per requirement
MTM_data['Status'].replace('Already', 'Already_Matched', inplace= True)
MTM_data['Status'].replace('Not', 'Not_Worked', inplace = True)
MTM_data['Status'].unique()

In [ ]:
#Make a key of By and Status column
MTM_data['key'] = MTM_data['Status'].astype(str) + '_' +MTM_data['By'].astype(str)
MTM_data.head()

In [ ]:
#Generate the random number against each title and sort the
random.seed(42)
MTM_data['Rnos'] = [random.random() for k in MTM_data.index]
MTM_data.sort_values(['key','Rnos'],ascending=[True, False],inplace=True)
MTM_data.head()

In [ ]:
#count each category belongs to 'key' column and put that in the new column
MTM_data['Total_Tiltes_worked'] = MTM_data.groupby('key')['key'].transform('count')


In [ ]:
#Make a new dataframe which gives the unique values in key and Total_Tiltes_worked columns
distribution_of_key=MTM_data.drop_duplicates(['key','Total_Tiltes_worked'])[['key','Total_Tiltes_worked']]
distribution_of_key

In [ ]:
#splitting the key to get the Already status seperately
distribution_of_key['status_first_word'] = distribution_of_key['key'].str.split('_').str[0]
distribution_of_key

In [ ]:
#Assigning percentages against each key
distribution_of_key['percentage'] = np.where(distribution_of_key['status_first_word']=='Already', 0.05, percent)
distribution_of_key

In [ ]:
#Calculating the exact number of titles for each key for assigned percentage
distribution_of_key['20percent_of_total_titles'] = round(distribution_of_key['Total_Tiltes_worked']*distribution_of_key['percentage'],ndigits=0)
distribution_of_key['20percent_of_total_titles'].replace(0,1,inplace = True)
distribution_of_key

In [ ]:
#Merging the main data(MTM_data) with distribution data to get the 20percent of titles in the main data
MTM_data = pd.merge(MTM_data,distribution_of_key[['key','20percent_of_total_titles']], how='left', on='key')
MTM_data.head()

In [ ]:
#making a new column 'count' where we are distributing total_titles_worked column based on the values in the column from 1 to n where n is the count of key in the dataframe
MTM_data['count'] = 1
for i in range(len(MTM_data)-1):
    if MTM_data.ix[i+1,'key'] == MTM_data.ix[i,'key']:
        MTM_data.ix[i+1,'count'] = MTM_data.ix[i,'count']+1
    else:
        MTM_data.ix[i+1,'count']

In [ ]:
MTM_data

In [ ]:
#Adding column of which titles to check for peer review
MTM_data['Titles_to_check'] = MTM_data.apply(lambda x: 'to_recheck' if x['count'] <= x['20percent_of_total_titles'] else 'not to_recheck', axis = 1)

In [ ]:
#Selecting rows with only 'to_recheck' status and making the final data
MTM_data_final = MTM_data[MTM_data['Titles_to_check'] == 'to_recheck']
print('Dimensions of MTM_data_final:',MTM_data_final.shape)

In [ ]:
#Count total already matched titles
distribution_of_key_already_matched = distribution_of_key[distribution_of_key['status_first_word'] == 'Already']
Total_Already_Matched_Titles = round((distribution_of_key_already_matched['20percent_of_total_titles'].sum()),ndigits=0)
print('Total Already Matched Titles:',Total_Already_Matched_Titles)

In [ ]:
#Count total rest matched titles
distribution_of_key_rest_matched = distribution_of_key[distribution_of_key['status_first_word'] != 'Already']
Total_Rest_Matched_Titles = round((distribution_of_key_rest_matched['20percent_of_total_titles'].sum()),ndigits=0)
print('Total Rest Matched Titles:',Total_Rest_Matched_Titles)

In [ ]:
#Count total titles to recheck
Total_Titles_to_recheck = Total_Already_Matched_Titles + Total_Rest_Matched_Titles
print('Total_Titles_to_recheck:',Total_Titles_to_recheck)

In [ ]:
#QC of total number of titles to recheck in the final dataframe and total number of titles to recheck counted from the individual distribution of percentages for each key
QC = len(MTM_data_final) - Total_Titles_to_recheck
if QC == 0:
    print('Correct calculation')
else:
    print('Error in the calculation')

In [ ]:
MTM_data_final=MTM_data_final.drop(['Rnos','Total_Tiltes_worked','key','20percent_of_total_titles','count','Titles_to_check','Status' ], axis=1)

In [ ]:
#Export final data which need to recheck
MTM_data_final.to_csv(path+'/'+file_name+'_verify.csv')

In [ ]:
assign=MTM_data_final.sort_values("By")
assign= assign.groupby(['By']) 

#MTM_data_final[MTM_data_final["By"]=="SP"].nunique()

In [ ]:
#Separating each members data
names={}
for i in reviewers:
    df=assign.get_group(i)
    names.update({i:df})

In [ ]:
#Dividing each members data so that it can distributed equally among other members for verification
#SP['Rnos'] = 0
members=names.values()
num=[]
for i in range(len(members)):
    num.append(i)
del num[0]
c=0
for j in members:
    for i in j.index :
        j.loc[i, "Rnos"]=num[c]
        if c==len(num)-1:
            c=0
        else:
            c=c+1 
    
print(names)

In [ ]:
#function which creates verification file for each member on passing the names dictionary as argument

def verify(names):
    counter={}
    names_set=set(names)
    
    #Initialising counter dictionary with key as member name and value as 1
    for val in names:
        temp={val:1}
        counter.update(temp)
       
        
    
    for person, data in names.items() :
        
        verify_file_name=person
        verifier={person}
        verification_set=names_set-verifier
        print(verification_set)
        #verify dictionary has verifier name as key and verification names as values
        verify={person:verification_set}
        print(verify)
        verification_file=pd.DataFrame()
        print('*****************')
        for people in verification_set :
            print(people)
            counter_val=counter[people]
            people_data=names[people]
            person=people_data[people_data["Rnos"]==counter_val]
            #print(person, each, counter_val)
            verification_file=verification_file.append(person)
            verification_file.to_csv(path+'/'+file_name+'_'+str(verify_file_name)+'_verify.csv')
            counter_val=counter_val+1
            up={people:counter_val}
            counter.update(up)
            

            
verify(names)
